In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
import torch.nn.functional as F

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
traveller_folder = '/home/work/recsys/travel/data/csv/traveller/'
sgg_folder = '/home/work/recsys/travel/data/csv/sgg/'
travel_folder = '/home/work/recsys/travel/data/csv/travel/'
area_folder = '/home/work/recsys/travel/data/csv/area/'

In [3]:
df_traveller_A = pd.read_csv(os.path.join(traveller_folder, 'train', 'tn_traveller_master_A.csv'))
df_traveller_B = pd.read_csv(os.path.join(traveller_folder, 'train', 'tn_traveller_master_B.csv'))
df_traveller_C = pd.read_csv(os.path.join(traveller_folder, 'train', 'tn_traveller_master_C.csv'))
df_traveller_D = pd.read_csv(os.path.join(traveller_folder, 'train', 'tn_traveller_master_D.csv'))

df_travel_A = pd.read_csv(os.path.join(travel_folder, 'train', 'tn_travel_A.csv'))
df_travel_B = pd.read_csv(os.path.join(travel_folder, 'train', 'tn_travel_B.csv'))
df_travel_C = pd.read_csv(os.path.join(travel_folder, 'train', 'tn_travel_C.csv'))
df_travel_D = pd.read_csv(os.path.join(travel_folder, 'train', 'tn_travel_D.csv'))

df_area_A = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_A.csv'))
df_area_B = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_B.csv'))
df_area_C = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_C.csv'))
df_area_D = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_D.csv'))

df_traveller = pd.concat([df_traveller_A, df_traveller_B, df_traveller_C, df_traveller_D]).reset_index(drop=True)
df_travel = pd.concat([df_travel_A, df_travel_B, df_travel_C, df_travel_D]).reset_index(drop=True)
df_area = pd.concat([df_area_A, df_area_B, df_area_C, df_area_D]).reset_index(drop=True)


/tmp/ipykernel_154712/985280166.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_area_B = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_B.csv'))
/tmp/ipykernel_154712/985280166.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_area_C = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_C.csv'))
/tmp/ipykernel_154712/985280166.py:14: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_area_D = pd.read_csv(os.path.join(area_folder, 'train', 'tn_visit_area_info_D.csv'))


In [4]:
def area_preprocess(df_area):
    df_area = df_area[(df_area['VISIT_AREA_TYPE_CD'] == 1) | (df_area['VISIT_AREA_TYPE_CD'] == 2) |
                  (df_area['VISIT_AREA_TYPE_CD'] == 3) | (df_area['VISIT_AREA_TYPE_CD'] == 4) |
                  (df_area['VISIT_AREA_TYPE_CD'] == 5) | (df_area['VISIT_AREA_TYPE_CD'] == 6) |
                  (df_area['VISIT_AREA_TYPE_CD'] == 7) | (df_area['VISIT_AREA_TYPE_CD'] == 8) ]
    df_area.dropna(subset = ['LOTNO_ADDR'], inplace = True)
    df_area = df_area.reset_index(drop = True)
    
    sido = []
    gungu = []
    
    for i in range(len(df_area['LOTNO_ADDR'])):
        sido.append(df_area['LOTNO_ADDR'][i].split(' ')[0])
        gungu.append(df_area['LOTNO_ADDR'][i].split(' ')[1])
    df_area['SIDO'] = sido
    df_area['GUNGU'] = gungu
    
    area_feature = ['TRAVEL_ID', 'VISIT_AREA_NM', 'VISIT_ORDER', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'X_COORD', 'Y_COORD',
                   'VISIT_START_YMD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN'
    ]
    df_area = df_area[area_feature]
    
    return df_area
    
df_area = area_preprocess(df_area)

/tmp/ipykernel_154712/3617364338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area.dropna(subset = ['LOTNO_ADDR'], inplace = True)


In [5]:
df_area[df_area['TRAVEL_ID'] == 'a_a000007']

,TRAVEL_ID,VISIT_AREA_NM,VISIT_ORDER,SIDO,GUNGU,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,VISIT_START_YMD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN
40847,a_a000007,플로망스 수영장,6,인천,강화군,6,126.3766478,37.6245423,2022-08-20,1.0,5.0,4.0,420.0,Y
42377,a_a000007,보문사,3,인천,강화군,2,126.3214866,37.68881563,2022-08-19,4.0,3.0,4.0,30.0,N


In [6]:
def travel_preprocess(df):
    travel_list = []
    for i in range (len(df)):
        check = df['TRAVEL_MISSION_CHECK'][i]
        priority = int(check.split(';')[0])
        travel_list.append(priority)
    df['TRAVEL_MISSION_PRIORITY'] = travel_list
    
    # 날짜 열을 datetime 형식으로 변환
    df['TRAVEL_START_YMD'] = pd.to_datetime(df['TRAVEL_START_YMD'])
    df['TRAVEL_END_YMD'] = pd.to_datetime(df['TRAVEL_END_YMD'])

    # 여행 일수 계산 (여행 종료일 - 여행 시작일) + 1
    df['TRAVEL_DAYS'] = (df['TRAVEL_END_YMD'] - df['TRAVEL_START_YMD']).dt.days + 1
    df_travel = df[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY', 'TRAVEL_DAYS']]
    
    return df_travel

df_travel = travel_preprocess(df_travel)
df_travel

,TRAVEL_ID,TRAVELER_ID,TRAVEL_MISSION_PRIORITY,TRAVEL_DAYS
0,a_a014262,a014262,2,2
1,a_a006728,a006728,6,2
2,a_a015745,a015745,13,4
3,a_a010544,a010544,1,2
4,a_a012058,a012058,1,2
...,...,...,...,...
12795,b_b001198,b001198,25,3
12796,d_d009222,d009222,23,3
12797,d_d002847,d002847,21,4
12798,d_d003147,d003147,21,3


In [7]:
def traveller_preprocess(df_traveller):
    traveller_feature = [
        'TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME',
        'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
        'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM'
    ]
    df_traveller = df_traveller[traveller_feature]
    
    return df_traveller

df_traveller = traveller_preprocess(df_traveller)

In [8]:
df = pd.merge(df_travel, df_traveller, left_on='TRAVELER_ID', right_on='TRAVELER_ID', how='inner')
df = pd.merge(df_area, df, left_on='TRAVEL_ID', right_on='TRAVEL_ID', how='right')


In [9]:
def preprocess_values(df):
    df = df.dropna(subset = ['TRAVEL_STYL_1', 'TRAVEL_MOTIVE_1', 'DGSTFN'])
    df.reset_index(drop = True, inplace = True)

    df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].replace(0,60)
    df['REVISIT_YN'] = df['REVISIT_YN'].replace('N', 0)
    df['REVISIT_YN'] = df['REVISIT_YN'].replace('Y', 1)
    df['GENDER'] = df['GENDER'].replace('남', 0)
    df['GENDER'] = df['GENDER'].replace('여', 1)
    
    #EMBEDDING 위해 0부터 시작하려고 1 감소
    df['TRAVEL_STYL_1'] = df['TRAVEL_STYL_1'] - 1
    df['TRAVEL_STYL_2'] = df['TRAVEL_STYL_2'] - 1
    df['TRAVEL_STYL_3'] = df['TRAVEL_STYL_3'] - 1
    df['TRAVEL_STYL_4'] = df['TRAVEL_STYL_4'] - 1
    df['TRAVEL_STYL_5'] = df['TRAVEL_STYL_5'] - 1
    df['TRAVEL_STYL_6'] = df['TRAVEL_STYL_6'] - 1
    df['TRAVEL_STYL_7'] = df['TRAVEL_STYL_7'] - 1
    df['TRAVEL_STYL_8'] = df['TRAVEL_STYL_8'] - 1
    df['TRAVEL_MOTIVE_1'] = df['TRAVEL_MOTIVE_1'] - 1
    
    # 'SIDO'가 '광복동' 또는 '동부리'인 행 삭제
    df = df[~df['SIDO'].isin(['광복동', '동부리'])]

    # 이름 통일을 위한 매핑 딕셔너리
    name_mapping = {
        '강원도': '강원',
        '경기도': '경기',
        '경상남도': '경남',
        '경상북도': '경북',
        '광주광역시': '광주',
        '대구광역시': '대구',
        '대전광역시': '대전',
        '부산광역시': '부산',
        '서울특별시': '서울',
        '세종특별자치시': '세종',
        '울산광역시': '울산',
        '인천광역시': '인천',
        '전라남도': '전남',
        '전라북도': '전북',
        '충청남도': '충남',
        '충청북도': '충북',
        '제주특별자치도': '제주'
    }

    # 이름 통일 적용
    df['SIDO'] = df['SIDO'].map(name_mapping).fillna(df['SIDO'])
    
    return df

df = preprocess_values(df)


/tmp/ipykernel_154712/234024890.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].replace(0,60)
/tmp/ipykernel_154712/234024890.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['REVISIT_YN'] = df['REVISIT_YN'].replace('N', 0)
/tmp/ipykernel_154712/234024890.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
features = [
    'TRAVEL_ID','TRAVELER_ID', 'VISIT_AREA_NM', 'VISIT_ORDER', 'VISIT_AREA_TYPE_CD',
    'VISIT_START_YMD', 'TRAVEL_DAYS', 'SIDO', 'GUNGU', 'X_COORD', 'Y_COORD',
    'RESIDENCE_TIME_MIN', 'REVISIT_YN', 'TRAVEL_MISSION_PRIORITY',
    'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
    'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM',
    'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION'
]

df = df[features]

df

,TRAVEL_ID,TRAVELER_ID,VISIT_AREA_NM,VISIT_ORDER,VISIT_AREA_TYPE_CD,VISIT_START_YMD,TRAVEL_DAYS,SIDO,GUNGU,X_COORD,...,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
0,a_a014262,a014262,DMZ 장단콩두부마을,2.0,7.0,2022-10-29,2,경기,파주시,126.689648,...,1,5,0,6,2.0,2,1,5.0,4.0,4.0
1,a_a014262,a014262,파주프리미엄아울렛,6.0,4.0,2022-10-29,2,경기,파주시,126.696335,...,1,5,0,6,2.0,2,1,4.0,4.0,4.0
2,a_a014262,a014262,한국근현대사박물관,5.0,2.0,2022-10-29,2,경기,파주시,126.695182,...,1,5,0,6,2.0,2,1,3.0,2.0,2.0
3,a_a006728,a006728,벨라시타,2.0,4.0,2022-10-10,2,경기,고양시,126.791452,...,4,4,4,4,5.0,1,3,4.0,4.0,4.0
4,a_a006728,a006728,이케아 고양점,4.0,4.0,2022-10-10,2,경기,고양시,126.863134,...,4,4,4,4,5.0,1,3,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28618,d_d003147,d003147,올레길 20코스(김녕-하도 올레),5.0,7.0,2022-10-29,3,제주,제주시,126.805472,...,1,1,4,3,1.0,2,0,4.0,4.0,4.0
28619,d_d003147,d003147,지미봉,12.0,1.0,2022-10-30,3,제주,제주시,126.902439,...,1,1,4,3,1.0,2,0,5.0,4.0,4.0
28620,d_d003147,d003147,월정리해수욕장,3.0,1.0,2022-10-29,3,제주,제주시,126.795805,...,1,1,4,3,1.0,2,0,4.0,4.0,4.0
28621,d_d003147,d003147,세화민속오일시장,9.0,4.0,2022-10-30,3,제주,제주시,126.8574647,...,1,1,4,3,1.0,2,0,4.0,4.0,4.0


In [11]:
# 'VISIT_START_YMD'를 datetime으로 변환
df['VISIT_START_YMD'] = pd.to_datetime(df['VISIT_START_YMD'])

# 'TRAVEL_ID'와 'VISIT_START_YMD'로 정렬
df.sort_values(by=['TRAVEL_ID', 'VISIT_START_YMD', 'VISIT_ORDER'], inplace=True)

# 각 여행(TRAVEL_ID)의 시작 날짜를 기준으로 일차(Day) 계산
df['DAY'] = df.groupby('TRAVEL_ID')['VISIT_START_YMD'].transform(lambda x: (x - x.min()).dt.days + 1)

# 각 일차별로 VISIT_ORDER 재정렬
df['VISIT_SEQUENCE'] = df.groupby(['TRAVEL_ID', 'DAY']).cumcount() + 1



In [12]:
df.sort_values(by=['TRAVEL_ID', 'VISIT_START_YMD', 'VISIT_ORDER'], inplace=True)

# 날짜별로 데이터 분할 및 시퀀스 데이터 생성
grouped = df.groupby(['TRAVEL_ID', 'VISIT_START_YMD'])

sequences_df = df.groupby(['TRAVEL_ID', 'DAY']).apply(lambda x: x.sort_values(by=['VISIT_SEQUENCE']))
sequences_df.columns

Index(['TRAVEL_ID', 'TRAVELER_ID', 'VISIT_AREA_NM', 'VISIT_ORDER',
       'VISIT_AREA_TYPE_CD', 'VISIT_START_YMD', 'TRAVEL_DAYS', 'SIDO', 'GUNGU',
       'X_COORD', 'Y_COORD', 'RESIDENCE_TIME_MIN', 'REVISIT_YN',
       'TRAVEL_MISSION_PRIORITY', 'GENDER', 'AGE_GRP', 'INCOME',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM', 'DGSTFN',
       'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'DAY', 'VISIT_SEQUENCE'],
      dtype='object')

In [13]:
features = [
    'TRAVEL_ID','TRAVELER_ID', 'VISIT_AREA_NM', 'DAY', 'VISIT_SEQUENCE', 'VISIT_AREA_TYPE_CD',
    'VISIT_START_YMD', 'SIDO', 'GUNGU', 'X_COORD', 'Y_COORD',
    'RESIDENCE_TIME_MIN', 'REVISIT_YN', 'TRAVEL_MISSION_PRIORITY',
    'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
    'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM',
    'DGSTFN'
]

categorical_features = [
    'TRAVEL_ID','TRAVELER_ID', 'VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD',
    'VISIT_START_YMD', 'SIDO', 'GUNGU', 'TRAVEL_MISSION_PRIORITY', 'AGE_GRP', 'INCOME',
    'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1'    
]

continuous_features = [
    'DAY', 'VISIT_SEQUENCE', 'X_COORD', 'Y_COORD', 'RESIDENCE_TIME_MIN', 'REVISIT_YN', 'GENDER',
    'TRAVEL_COMPANIONS_NUM', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
    'DGSTFN'
]


In [14]:
from torch.utils.data import Dataset, DataLoader
import torch

class TravelDataset(Dataset):
    def __init__(self, dataframe, categorical_features, continuous_features):
        self.dataframe = dataframe
        self.categorical_data = dataframe[categorical_features].astype('category').apply(lambda x: x.cat.codes).values
        self.continuous_data = dataframe[continuous_features].values
        self.labels = dataframe['VISIT_AREA_NM'].astype('category').cat.codes 

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        cat_data = torch.tensor(self.categorical_data[idx], dtype=torch.long)
        cont_data = torch.tensor(self.continuous_data[idx], dtype=torch.float)
        label = self.labels[idx]
        return cat_data, cont_data, label

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiRNNWithEmbedding(nn.Module):
    def __init__(self, embedding_dims, num_continuous, rnn_hidden_size, num_layers, output_size):
        super(BiRNNWithEmbedding, self).__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(num_embeddings, embedding_dim) for num_embeddings, embedding_dim in embedding_dims])
        self.rnn = nn.GRU(sum(embedding_dims) + num_continuous, rnn_hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(rnn_hidden_size * 2, output_size)

    def forward(self, x_cat, x_cont):
        x_embedded = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_embedded = torch.cat(x_embedded, dim=1)
        x_combined = torch.cat((x_embedded, x_cont), dim=1)
        rnn_out, _ = self.rnn(x_combined.unsqueeze(1))
        output = self.fc(rnn_out.squeeze(1))
        return output



In [16]:
import torch.optim as optim

dataset = TravelDataset(sequences_df, categorical_features, continuous_features)

data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델 인스턴스화
embedding_dims = [(sequences_df[feature].nunique(), min(50, (sequences_df[feature].nunique()+1)//2)) for feature in categorical_features]
num_classes = sequences_df['VISIT_AREA_NM'].nunique()
num_continuous = len(continuous_features)

model = BiRNNWithEmbedding(embedding_dims, num_continuous, rnn_hidden_size=128, num_layers=2, num_classes=num_classes)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 디바이스로 이동
model.to(device)

# 학습 과정
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (cat_data, cont_data, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        
        # GPU 사용 가능한 경우 데이터를 GPU로 이동
        cat_data = cat_data.to(device)
        cont_data = cont_data.to(device)
        labels = labels.to(device)
        
        # 모델에 x_lengths도 전달
        outputs = model(cat_data, cont_data, x_lengths)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0




TypeError: __init__() got an unexpected keyword argument 'num_classes'

In [ ]:
filtered_df = sequences_df[sequences_df['SIDO'] == '경기']
num_classes = filtered_df['VISIT_AREA_NM'].nunique()
n = sequences_df['SIDO'].nunique()
sido_row_counts = sequences_df.groupby('SIDO').size()
sido_row_counts

In [ ]:
#model.eval()  # 모델을 평가 모드로 설정
#with torch.no_grad():  # 그라디언트 계산을 비활성화

